## SegVol inference

In [2]:
from adapt_med_seg.pipelines.evaluate import EvaluateArgs, EvaluatePipeline

model_names = ["segvol_baseline"]
dataset_numbers = [0]

for model_name in model_names:
    for dataset_number in dataset_numbers:
        eval_pipeline = EvaluatePipeline(
            evaluate_args=EvaluateArgs(
                dataset_number=dataset_number,
                model_name=model_name,
                cls_idx=0,
                device='cpu'
                # use_wandb=True,
            )
        )

        results = eval_pipeline.run()
        print(f'Results for {model_name} on dataset {dataset_number}:\n{results}')

/Users/Matey/miniconda3/envs/SegVol/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


File /Users/Matey/project/adapt_med_seg/datasets/0000.zip created.


71093it [00:58, 1220.00it/s]                           


Extracting /Users/Matey/project/adapt_med_seg/datasets/0000.zip


INFO:adapt_med_seg.pipelines.evaluate:Evaluating segvol_baseline on dataset 0


Files extracted to /Users/Matey/project/adapt_med_seg/datasets
File downloaded and saved as 0000


Evaluating CHAOS: 100%|██████████| 4/4 [06:27<00:00, 96.92s/batch] 

Results for segvol_baseline on dataset 0:
{'dice': tensor(0.9659)}


# SegVol vanilla training

In [ ]:
from adapt_med_seg.data.dataset import MedSegDataset
from adapt_med_seg.pipelines.train import SegVolLightning, TrainingArgs

from pytorch_lightning import Trainer, seed_everything

seed_everything(42)

model_names = ["segvol_baseline"]
dataset_numbers = [0]

for model_name in model_names:
    for dataset_number in dataset_numbers:
        training_args = TrainingArgs(
            dataset_number=dataset_number,
            model_name=model_name,
            cls_idx=0,
            device="cpu",
            # use_wandb=True,
        )

        train_pipeline = SegVolLightning(model_name)

        _dataset = MedSegDataset(
            dataset_number=dataset_number,
            processor=train_pipeline._model.processor,
            train=True,
        )

        train_pipeline.set_dataset(_dataset)
        train_dataloader, val_dataloader = _dataset.get_train_val_dataloaders(
            1 / 9, 1, 42
        )
        
        trainer = Trainer(
            max_epochs=10,
            accelerator=training_args.device,
            deterministic=True,
            num_sanity_val_steps=0,
        )
        trainer.fit(train_pipeline, train_dataloader, val_dataloader)